In [11]:
%load_ext nb_black
# редактирование ячеек по стандартам pep8

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## Загрузка парсеров и sklearn

In [2]:
!python -m spacy download ru_core_news_sm

     -------------------------------------- 15.3/15.3 MB 391.4 kB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


You should consider upgrading via the 'C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [30]:
!pip install natasha

     ---------------------------------------- 34.4/34.4 MB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 41.1/41.1 KB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 49.4/49.4 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=1c7333882dcbcccd2f4d49a599a41138fe7edeaad4dd105574dff0bb54da5c6f
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\fa\80\8c\43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


You should consider upgrading via the 'C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [15]:
!pip install pymorphy2

You should consider upgrading via the 'C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [41]:
!pip install sklearn

In [3]:
from sklearn.metrics import accuracy_score

<IPython.core.display.Javascript object>

## О создании корпуса

В корпус я включала преимущественно предложения, в которых нужно разрешать грамматическую омонимию (а именно нужно верно отнести слова вроде "мой", "стали", "течь" и т. д. к нужной части речи, что может составлять трудность для парсеров) (я просто гуглила что-то вроде "примеры грамматической омонимии" и "упражнения на разрешение грамматической омонимии для школьников" и вставляла в текстовый файл то, что подходит именно для проверки правильности частеречной разметки). 

## Парсинг

Парсинг с помощью SpaCy

In [38]:
import spacy
import pandas as pd

<IPython.core.display.Javascript object>

In [5]:
with open("corpus.txt", "r", encoding="utf-8") as f:
    text = f.read()

nlp = spacy.load("ru_core_news_sm")
doc = nlp(text)

<IPython.core.display.Javascript object>

In [6]:
tokens = []
pos = []
for token in doc:
    tokens.append(token)
    pos.append(token.pos_)

list_tuples = list(zip(tokens, pos))
df = pd.DataFrame(list_tuples, columns=["Tokens", "POS"])
df.to_csv("pos_spacy", sep="\t")

<IPython.core.display.Javascript object>

Парсинг с помощью pymorphy2

In [7]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

<IPython.core.display.Javascript object>

In [8]:
tokens = []
pos = []

for token in doc:
    tokens.append(token.text)
    pos.append(morph.parse(token.text)[0].tag.POS)

list_tuples = list(zip(tokens, pos))
df = pd.DataFrame(list_tuples, columns=["Tokens", "POS"])
df.to_csv("pos_pymorphy", sep="\t")

<IPython.core.display.Javascript object>

Парсинг с помощью natasha

In [9]:
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, Doc

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

<IPython.core.display.Javascript object>

In [12]:
doc_2 = Doc(text)
doc_2.segment(segmenter)
doc_2.tag_morph(morph_tagger)

tokens = []
pos = []
for token in doc_2.tokens:
    tokens.append(token.text)
    pos.append(token.pos)

list_tuples = list(zip(tokens, pos))
df = pd.DataFrame(list_tuples, columns=["Tokens", "POS"])
df.to_csv("pos_natasha", sep="\t")

<IPython.core.display.Javascript object>

## Конвертация тегов в один стандарт

В ручной разметке я воспользовалась немного изменённым инвентарём Universal Dependencies (все союзы разметила тегом CONJ, не размечала отдельно имена собственные, никакие глаголы не размеяала как вспомогательные, т. е. тегом AUX). За основу взяла именно Universal Dependencies, а не OpenCorpora, потому что этот инвентарь, во-первых, используется гораздо шире, а во-вторых, меньше, чем OpenCorpora. Это, на мой взгляд, преимущество этого инвентаря для данной задачи, потому что нам больше хочется посмотреть на то, отличит ли парсер существительное / местоимение от глагола, чем на то, отличит ли он инфинитив от неинфинитива.

###### Функция конвертации для spacy и natasha

Так как при ручной разметке ориентировалась на UD, здесь всё просто

In [13]:
def ud_tags_converter(file):
    tags_dict = {"PROPN": "NOUN", "AUX": "VERB", "SCONJ": "CONJ", "CCONJ": "CONJ"}
    with open(file, "r", encoding="utf-8") as f:
        text = f.read()
        for key in tags_dict:
            text = text.replace(key, tags_dict[key])
    with open(file, "w", encoding="utf-8") as f:
        f.write(text)

<IPython.core.display.Javascript object>

In [14]:
ud_tags_converter("pos_natasha")

<IPython.core.display.Javascript object>

In [15]:
ud_tags_converter("pos_spacy")

<IPython.core.display.Javascript object>

###### Функция конвертации для pymorphy

Здесь нужно не просто заменить теги, нужно также отловить притяжательные местоимения, которые размечаются тем же тегом, что и полные имена прилагательные

In [16]:
def oc_tags_converter(file):
    tags_dict = {
        "ADJF": "ADJ",
        "ADJS": "ADJ",
        "COMP": "ADJ",
        "INFN": "VERB",
        "PRTF": "VERB",
        "PRTS": "VERB",
        "GRND": "VERB",
        "NUMR": "NUM",
        "ADVB": "ADV",
        "NPRO": "PRON",
        "PRED": "ADV",
        "PREP": "ADP",
        "PRCL": "PART",
    }
    with open(file, "r", encoding="utf-8") as f:
        text = f.readlines()
        for i in range(len(text)):  # смена тега у местоимённых прилагательных
            if "ADJF" in text[i]:
                if (
                    "Apro" in morph.parse(text[i].split("\t")[1])[0].tag
                ):  # Apro — граммема для местоимённых прилагательных
                    text[i] = text[i].replace("ADJF", "DET")
            for key in tags_dict:  # смена остальных тегов
                try:
                    text[i] = text[i].replace(key, tags_dict[key])
                except:
                    continue
    with open(file, "w", encoding="utf-8") as f:
        f.write("".join(text))

<IPython.core.display.Javascript object>

In [17]:
oc_tags_converter("pos_pymorphy")

<IPython.core.display.Javascript object>

## Подсчёт accuracy

In [18]:
from sklearn.metrics import accuracy_score

<IPython.core.display.Javascript object>

In [19]:
def accuracy(file):
    manual_tagging_df = pd.read_csv("pos_tagging.csv", sep=",")
    manual_tagging = list(manual_tagging_df["POS"])
    parser_tagging_df = pd.read_csv(file, sep="\t")
    parser_tagging = list(parser_tagging_df["POS"])
    print("Accuracy: %.4f" % accuracy_score(parser_tagging, manual_tagging))

<IPython.core.display.Javascript object>

In [20]:
accuracy("pos_natasha")

Accuracy: 0.8607


<IPython.core.display.Javascript object>

In [21]:
accuracy("pos_spacy")

Accuracy: 0.9000


<IPython.core.display.Javascript object>

In [22]:
accuracy("pos_pymorphy")

Accuracy: 0.6821


<IPython.core.display.Javascript object>

Лучше всего с задачей справляется SpaCy

## Чанкер

Мы будем выделять следующие типы n-грамм:
1. ADV + ADJ (very nice), так как если прилагательное используется вместе с наречием, то, скорее всего, тональность прилагательного усиливается, что было бы хорошо учесть (вероятно, будет отдельно считаться и прилагательное, и сочетание "наречие + это же прилагательное", что, на мой взгляд, далеко не минус)
2. Not + ADJ (not exciting), так как частица not меняет смысл на противоположный
3. Not + VERB ((does) not work), так как можно дать оценку чему-то, используя глагол с отрицательной частицей (достаточно сказать "игра не запускается", чтобы это можно было считать отрицательным отзывом)

В качестве парсера будем использовать spacy

In [23]:
nlp = spacy.load("en_core_web_sm")

<IPython.core.display.Javascript object>

Внесём изменения в функцию, отвечающую за препроцессинг:

In [24]:
def preprocessing_with_bigrams(
    list,
):  # мы работаем со списками, удобнее написать функцию, на вход которой можно сразу подать список
    preprocessed_words = []
    bigrams = {"ADJ": ["ADV", "not"], "VERB": "not"}
    for text in list:
        preprocessed_text = nlp(
            text.lower()
        )  # здесь текст уже делится по предложениям и токенизируется
        tokens = [token for token in preprocessed_text if not token.is_punct]
        for i in range(len(tokens)):
            if tokens[i].lemma_.isalpha:
                preprocessed_words.append(tokens[i].lemma_)
            if tokens[i].pos_ in bigrams:
                if (
                    tokens[i - 1].pos_ in bigrams[tokens[i].pos_]
                    or tokens[i - 1].lemma_ in bigrams[tokens[i].pos_]
                ):
                    preprocessed_words.append(
                        tokens[i - 1].lemma_ + " " + tokens[i].lemma_
                    )
    return preprocessed_words

<IPython.core.display.Javascript object>

Пробуем снова посчитать тональности отзывов, запускаем код из предыдущей домашки:

In [25]:
import requests
import time
import random
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

positive = []
negative = []

session = requests.session()

ua = UserAgent(verify_ssl=False)
headers = {"User-Agent": ua.random}
random.uniform(1, 3)

for num in range(19):
    for _ in range(5):
        response = session.get(
            f"https://www.metacritic.com/game/pc/minecraft/user-reviews?page={num}",
            headers=headers,
        )
        time.sleep(random.uniform(1.1, 5.2))

    page = response.text
    soup = BeautifulSoup(page, "html.parser")
    page_text = soup.find_all("div", {"class": "review_section"})

    for i in page_text:
        try:  # избегаем ошибки None has no attribute text, когда встречаем смешанные отзывы
            if i.find("div", {"class": "metascore_w user medium game positive indiv"}):
                try:
                    positive.append(
                        i.find("span", {"class": "blurb blurb_expanded"}).text
                    )  # если есть полная версия отзыва, пытаемся скачать её
                except:
                    positive.append(i.find("div", {"class": "review_body"}).text)
            elif i.find(
                "div", {"class": "metascore_w user medium game negative indiv"}
            ):
                try:
                    negative.append(
                        i.find("span", {"class": "blurb blurb_expanded"}).text
                    )  # если есть полная версия отзыва, пытаемся скачать её
                except:
                    negative.append(i.find("div", {"class": "review_body"}).text)
        except:
            continue

<IPython.core.display.Javascript object>

#### Уравниваем кол-во отзывов, откладывыем по 10 из каждого списка на тест

In [26]:
positive = positive[: len(negative)]  # уравниваем кол-во текстов

<IPython.core.display.Javascript object>

In [27]:
test = {}
for i in range(-1, -11, -1):
    test[positive.pop(i)] = "positive"
    test[negative.pop(i)] = "negative"

<IPython.core.display.Javascript object>

Считаем кол-во слов в каждом списке, убираем те, которые встречаются менее трёх раз

In [28]:
from collections import Counter

<IPython.core.display.Javascript object>

In [29]:
def deleting_unpopular(preprocessed_list):
    word_list = []
    cnt_dict = Counter(preprocessing_with_bigrams(preprocessed_list))
    for i in cnt_dict:
        if cnt_dict[i] > 2:
            word_list.append(i)
    return set(word_list)

<IPython.core.display.Javascript object>

Удаляем слова, которые есть и в позитивных, и в негативных отзывах

In [30]:
positive_set = deleting_unpopular(positive)

<IPython.core.display.Javascript object>

In [31]:
negative_set = deleting_unpopular(negative)

<IPython.core.display.Javascript object>

In [32]:
intersection = positive_set.intersection(negative_set)
positive_set.difference_update(intersection)
negative_set.difference_update(intersection)

<IPython.core.display.Javascript object>

Функция для определения тональности:

In [33]:
freq_lists = {}

freq_lists["positive"] = positive_set
freq_lists["negative"] = negative_set

<IPython.core.display.Javascript object>

In [34]:
def simple_sent_analysis(freq_lists, text):
    preprocessed_review_list = []
    preprocessed_review = nlp(
        text.lower()
    )  # здесь текст уже делится по предложениям и токенизируется
    for word in [token.lemma_ for token in preprocessed_review if not token.is_punct]:
        if word.isalpha:
            preprocessed_review_list.append(word)
    counts = Counter()
    for tone, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in preprocessed_review_list:
            counts[tone] += int(freq_list[word] > 0)
    return counts.most_common()[0][0]

<IPython.core.display.Javascript object>

Запущенная функция + тональность отзыва:

In [35]:
for i in test:
    print(simple_sent_analysis(freq_lists, i), test[i])

positive positive
negative negative
negative positive
negative negative
positive positive
negative negative
positive positive
negative negative
positive positive
positive negative
positive positive
negative negative
positive positive
negative negative
positive positive
positive negative
negative positive
positive negative
positive positive
negative negative


<IPython.core.display.Javascript object>

Подсчёт accuracy

In [36]:
def test_simple_sent_analysis(freq_lists, test_list):
    results = []
    gold = []
    for tone in freq_lists:
        for text in test_list:
            predicted_tone = simple_sent_analysis(freq_lists, text)
            results.append(predicted_tone)
            gold.append(test_list[text])
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))

<IPython.core.display.Javascript object>

In [37]:
test_simple_sent_analysis(freq_lists, test)

RESULTS:
Accuracy: 0.7500


<IPython.core.display.Javascript object>

Accuracy повысилась на 10%, ура!